In [137]:
import re
from collections import defaultdict
from string import ascii_lowercase

from scipy.sparse import csr_matrix, lil_matrix
from scipy.sparse.linalg import norm
import numpy as np
from sklearn.cluster import AgglomerativeClustering

In [175]:
stopwords = {'ltd', 'tel', 'fax', 'co', 'no', 'road', 'ul', 'sp', 'of', 'st', 'str', 'by', 'city', 'street', 'and', 'as', 'to', 'on', 'the'}
stopwords.update([str(x) for x in range(0, 10)])
stopwords.update(ascii_lowercase)

words = dict()
entries = []
vectors = []

with open("./lines.txt") as lines:
    # words = defaultdict(int)
    word_idx = 0
    for i, line in enumerate(lines):

        entry = {
            "id": i,
            "text": line,
        }

        tokens = list(filter(lambda x: x != "" and x not in stopwords, re.split(r'\W+', line.lower())))


        for token in tokens:
            if token not in words:
                words[token] = word_idx
                word_idx += 1

        vector = np.zeros(word_idx, dtype=np.int32)

        for token in tokens:
            vector[words[token]] += 1

        vector = csr_matrix(vector)
        vectors.append(vector)

        # if i == 266:
        #     print(line)
        #     print([(token, words[token]) for token in tokens])
        #     print(vector)
        #     break

        # tokens = [token for token in word_tokenize(line.lower(), language='english')]
        # print(line, " ".join(tokens))

        if i == 99: break

    # print(words)

# print(len(vectors))
n = len(vectors)
k = len(words)

matrix = lil_matrix((n, k), dtype=np.float32)

for i, vector in enumerate(vectors):
    vector.resize((1, k))
    vector = vector / norm(vector)
    matrix[i] = vector

matrix_T = matrix.T
matrix = matrix.tocsr()
matrix_T = matrix_T.tocsr()

# print(k, n)
# print(matrix)
# print(vectors[0].shape)
res = matrix @ matrix_T

dist = res.toarray()

np.savetxt("./matrix.txt", res.toarray(), fmt='%.2e')
cluster = AgglomerativeClustering(n_clusters=None, affinity='precomputed', connectivity=dist, linkage='average', distance_threshold=0.5)
cluster.fit_predict(dist)

    # print(stopwords)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_agglomerative.py:501: UserWarning: the number of connected components of the connectivity matrix is 2 > 1. Completing it to avoid stopping the tree early.
  connectivity, n_connected_components = _fix_connectivity(


ValueError: Precomputed metric requires shape (n_queries, n_indexed). Got (99, 100) for 1 indexed.

In [78]:
vector = np.zeros(12, dtype=np.int32)
vector[2] += 1
vector[5] += 2
vector[9] += 3
vector[11] += 10

# a = csr_matrix(([1, 2, 3, 10], ([0, 0, 0, 0], [2, 5, 9, 11])))
a = csr_matrix(vector)
b = csr_matrix(([5, 2, 1, 6], ([0, 0, 0, 0], [2, 5, 8, 11])))

In [79]:
print(a.toarray())
print(b.toarray())
c = a.dot(b.T)
print(c)

[[ 0  0  1  0  0  2  0  0  0  3  0 10]]
[[0 0 5 0 0 2 0 0 1 0 0 6]]
  (0, 0)	69


In [96]:
from sklearn.cluster import AgglomerativeClustering

matrix = lil_matrix((2, 12), dtype=np.float32)

matrix[0] = a
matrix[1] = b

dist = np.zeros((2, 2))
dist[0, 1] = a.dot(b.T)[0, 0]
dist[1, 0] = b.dot(a.T)[0, 0]

print(dist)

cluster = AgglomerativeClustering(affinity='precomputed', linkage='average')
cluster.fit_predict(dist)

[[ 0. 69.]
 [69.  0.]]


array([1, 0], dtype=int64)